In [1]:
# Import Libraries

import pyspark
from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
import json
import time
from pyspark.sql.functions import *
from pyspark.sql.types import *
from kafka import KafkaConsumer, TopicPartition

In [2]:
# Configuration

KAFKA_HOSTS = 'localhost:9092'
KAFKA_VERSION = (0, 10, 2)
topics = "posts"

In [3]:
# Spark

spark = SparkSession.builder.master("local[*]") \
    .appName("Profile Stream Consumer") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1") \
    .getOrCreate()


In [4]:
# Test Message

consumer = KafkaConsumer('test', bootstrap_servers=KAFKA_HOSTS, group_id=None, auto_offset_reset ='earliest')
tp = TopicPartition('test',0)
for message in consumer:
    print(message)
    if message.offset == consumer.position(tp) - 1:
        break

ConsumerRecord(topic='test', partition=0, offset=0, timestamp=1622090161916, timestamp_type=0, key=None, value=b'"Hello World!"', headers=[], checksum=None, serialized_key_size=-1, serialized_value_size=14, serialized_header_size=-1)


In [5]:
# Read Stream

lines = spark\
  .readStream\
  .format("kafka")\
  .option("kafka.bootstrap.servers", KAFKA_HOSTS)\
  .option("subscribe", topics)\
  .option("startingOffsets", "latest")\
  .load()

In [ ]:
query = lines\
 .selectExpr("CAST(value AS STRING)")\
 .writeStream\
 .format("csv")\
 .option("format", "append")\
 .option("path", "output/")\
 .option("checkpointLocation", "output/checkpoint/")\
 .outputMode("append")\
 .start()\
 .awaitTermination()

In [ ]:
spark.stop()